In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import requests,json

##Read Data <AS>
data = pd.read_excel('RO Integration.xlsx',sheet_name=0, converters={"Customer Code":str, "Service accounting codes":str,"GST Invoice":str,"Part HSN Code":str,"SAC Code":str, "Item_name":str})

In [2]:
#Cleaning Data
data = data.rename(columns=lambda x: x.strip())
data = data.replace(r'^\s*$', np.nan, regex=True).fillna(0)
data["Invoice_No"]=data["Invoice Number"]
data['Invoice_Date']= pd.to_datetime(data["Invoice Date"], dayfirst = True).dt.strftime('%d-%m-%Y')
seg_mapping = {
    'BAJD': 'KRPSC',
    'BAJG': 'BMRSC',
    'BAJM': 'SKBSC',
    'BAJP': 'YHKSC',
    'BAJS': 'KLNSC',
    'BAJV': 'SJPSC',
    'BAJY': 'YPRSC',
    'BALD': 'HLMSC',
    'TMJA': 'TMKSC',
    'BAJB': 'NGRAC',
    'BAJH': 'KLNBR'
}
data['Segment']=data['Dealer Code'].astype(str).map(seg_mapping)
data['GST_Rate'] = data['IGST %'].astype(int)+data['CGST %'].astype(int)+data['SGST %'].astype(int)
data['Sale_ledger1'] = np.where(data['Type'] == 'Labor', "Supply of Service @ "+ data['GST_Rate'].astype(str)+ '%', "Supply of Goods @ "+ data['GST_Rate'].astype(str)+ '%')
data['Vtype'] = "Service RO Bills - RI"+data['Segment'].astype(str)
data["party_ledger1"]=data["Registration No"].astype(str)+"-"+data["Customer Name"].astype(str)+"-"+data['Dealer Code'].astype(str)
data['IGST']=data['IGST AMT'].astype(float)
data['CGST']=data['CGST AMT'].astype(float)
data['SGST']=data['SGST AMT'].astype(float)
data["CESS"]= ''
data["CESS RATE"]=''
data['Discount'] = data['Discount %']
data["QTY"]= 1
data['Rate']=data["Net taxable Amount-Labour"].astype(float)+data["Net taxable Amount-Part"].astype(float)
data["unit"] = 'Nos'
data["Item_name"] = data['SAC Code'].astype(str) + data['Part HSN Code'].astype(str)
data['Amount']= data['Rate'].astype(float) - data['Discount']

In [3]:
data[['Invoice_No', 'Amount']]

,Invoice_No,Amount
0,SERVBAJD24007578,238.50
1,SERVBAJD24007578,50.00
2,SERVBAJD24007579,344.53
3,SERVBAJD24007579,127.12
4,SERVBAJD24007579,115.25
5,SERVBAJD24007579,42.19
6,SERVBAJD24007579,188.98
7,SERVBAJD24007579,273.44
8,SERVBAJD24007579,292.19
9,SERVBAJD24007579,557.00


In [4]:
data['Id'] = np.where(data['Amount']== 0,'Y','N')
data["party_ledger"] = np.where(data['Id']=='Y', 'Zero Invoice (E)',data["party_ledger1"])
data["Sale_ledger"] = np.where(data['Id']=='Y', 'Zero Invoice',data["Sale_ledger1"])
data['Narration'] = "Invoice Number-" + data["Invoice Number"].astype(str) + " | Invoice Date-" + data["Invoice Date"].astype(str) + "RO NO & DATE- " + data["Narration"].astype(str) + " | RO TYPE-" + data["RO TYPE"].astype(str) + "SA NAME- " + data["SA NAME"].astype(str) + " | Vin No.-" + data["VIN #"].astype(str) + "Vehicle No.- " + data["Registration No"].astype(str) + " | Vehcile Mode-" + data["MODEL"].astype(str)
roundOff = data.groupby(["Invoice_No"],as_index = False)[["Amount","IGST","SGST","CGST"]].sum()
roundOff["partyAmount"] = (roundOff["Amount"]+roundOff["IGST"]+roundOff["CGST"]+roundOff["SGST"]).round(2)
roundOff["round"] = (roundOff["partyAmount"].round(0)-roundOff["partyAmount"].round(2)).round(2)
data = data.merge(roundOff[['Invoice_No','round',"partyAmount"]], on=['Invoice_No'], how='left')
data.sort_values(by = ["Invoice_Date",'Invoice_No'],inplace=True)
data.reset_index(drop=True,inplace=True)

##All entry
# Empty List For 4 sheet
lst_voucher = []
lst_item = []
lst_gst = []
lst_round = []
lst_bills = []
lst_cc = []
lst_inv = []
lst_batchSpec = []
lst_multiDisc = []
lst_voucherProp = [] 


# First Number
num=1
isl=1
ccsl=1
lsl=1

# workings blank variable
var_vsl = ""
pre_vsl = ""
for ind in data.index:

    #Workings for Voucher SL and Item Sl
    if pre_vsl == "":
        pre_vsl = data['Invoice_No'][ind]

    elif pre_vsl != data['Invoice_No'][ind]:
        pre_vsl = data['Invoice_No'][ind]
        num = num+1
        isl = 1
    else:
        isl = isl+1


    # Workings on Voucher Sheet.
    if var_vsl != data['Invoice_No'][ind] :
        lst_voucher.append({
            '11v_sl_no' : num,
            '12v_date(dd-mm-yy)' : pd.to_datetime(data['Invoice_Date'][ind],dayfirst=True),
            '13vnum' : data['Invoice_No'][ind],
            '14vtype' : data['Vtype'][ind],
            '15narration' : data['Narration'][ind],
            '16sale_purchase_ledger' : data["Sale_ledger"][ind],
            '17party_ledger' : data['party_ledger'][ind],
            '18broker_name' : " ",
            '19Ref No' :data["Invoice_No"][ind],
            '20Ref Date' : pd.to_datetime(data['Invoice_Date'][ind],dayfirst=True),
            '21Fx (y/n)' : " ",
            '22Currency' : " ",
            '23Currency Rate' : " ",
            '24Branch' : '',
            '25Company GSTIN' : "29AADCT8379H1ZO",
            '26Challan No' : " ",
            '27Challan Date (dd-mm-yyy)' : " ",
            '28GST Party' :"",
            '29Delivery Date' : " ",
            '30Payment Term' : " ",
            '31Payment days' : " ",
            '32Freight Type' : " ",
            '33Reason For Issuing Document(As per GST)' : " ",
            '34Ref Receipt No' : " ",
            '35Ref Recript Date' : " ",
            '36Salesman' : "",
        })
        var_vsl = data['Invoice_No'][ind]
        
        #accaounting_part
        if data['round'][ind] !=0:   
            lst_round.append({
                '11v_sl_no' : num,
                '12ledger_name' : "Round Off",
                '13dr' : abs(data['round'][ind]) if data['round'][ind]<0 else 0,
                '14cr' : data['round'][ind] if data['round'][ind]>0 else 0,
                '15Ledger Ref No' : "",
                '16Fx Enable(y/n)' : "",
            })


    # Workings on Item Sheet.
    
    if data['Rate'][ind] !=0 :
        lst_item.append({
            '11v_sl_no' : num,
            '12item_s_no' : isl,
            '13Godown' : data['Segment'][ind],
            '14Item Name' : data['Item_name'][ind],
            '15Units(name)' : data["unit"][ind],
            '16Quantity' : data["QTY"][ind],
            '17Rate' : data['Rate'][ind],
            '18Discount (%)' : data['Discount'][ind],
            '19Amount (MRP)' : round(data['Amount'][ind],2),
            '20Batch' : '',
            '21Entry Type(consumption or production)' : " ",
            '22sale_purchase_ledger' : data["Sale_ledger"][ind],
            '23GRN Party Qty' : " ",
            '24GRN Qty' : " ",
            '25GRN Rej Qty' : " ",
            '26Order Ref No' : " ",
            '27Item Desc' : '',
            })
        
        lst_cc.append({
            '33Sl No' : ccsl,
            '34Voucher Sl No' : num,
            '35Cost Centre Date' : pd.to_datetime(data['Invoice_Date'][ind],dayfirst = True),
            '36Cost Centre / Cost Category Block 1' : data["Segment"][ind],
            '37Cost Centre Flag 1' : "c",
            '38Cost Centre / Cost Category Block 2' : " ",
            '39Cost Centre Flag 2' : " ",
            '43Amount' : round(data['Amount'][ind],2),
            '44Ledger Type (Sale/Purchase/Party)' : "Sale",
            '45Block No.' : 1,
            '46item_s_no' : isl,
        })             
        ccsl = ccsl+1    
        
        
        #GST SHeet 
        #IGST Calculation
        if data['IGST'][ind] > 0.0:
            lst_gst.append({
                '11v_sl_no' : num,
                '12item_s_no' : isl,
                '13Ledger Ref No' : " ",
                '14GST Ledger name' : f"IGST-Output @ {round((data['GST_Rate'][ind])/1,2)}%",
                '15Rate' : round((data['GST_Rate'][ind])/1,2),
                '16IGST' : data['IGST'][ind],
                '17CGST' : 0,
                '18SGST' : 0,
                '19Cess' : 0,
                '20Amount Type (cr/dr)' : "cr",
            })
        #CGST Calculation
        if data['CGST'][ind] > 0.0:
            lst_gst.append({
                '11v_sl_no' : num,
                '12item_s_no' : isl,
                '13Ledger Ref No' : " ",
                '14GST Ledger name' : f"CGST-Output @ {round((data['GST_Rate'][ind])/2,2)}%",
                '15Rate' : round((data['GST_Rate'][ind])/2,2),
                '16IGST' : 0,
                '17CGST' : data['CGST'][ind],
                '18SGST' : 0,
                '19Cess' : 0,
                '20Amount Type (cr/dr)' : "cr",
            })
        #SGST Calculation     
        if data['SGST'][ind] > 0.0:
            lst_gst.append({
                '11v_sl_no' : num,
                '12item_s_no' : isl,
                '13Ledger Ref No' : " ",
                '14GST Ledger name' : f"SGST-Output @ {round((data['GST_Rate'][ind])/2,2)}%",
                '15Rate' : round((data['GST_Rate'][ind])/2,2),
                '16IGST' : 0,
                '17CGST' : 0,
                '18SGST' : data['SGST'][ind],
                '19Cess' : 0,
                '20Amount Type (cr/dr)' : "cr",
            })

#preparingDataFrameForRealBooksUploadFormat
voucher = pd.DataFrame(lst_voucher)
if len(lst_voucher) > 0:
    voucher.columns = voucher.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_columns = ['v_sl_no','v_date(dd-mm-yy)','vnum', 'vtype', 'narration', 'sale_purchase_ledger','party_ledger','broker_name','Ref No','Ref Date','Fx (y/n)', 'Currency', 'Currency Rate', 'Branch', 'Company GSTIN']
    voucher = pd.DataFrame(columns=rb_iv_columns)

itemPart = pd.DataFrame(lst_item)
if len(lst_item) > 0:
    itemPart.columns = itemPart.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_itemPart = ['v_sl_no','item_s_no','Godown','Item Name','Units(name)','Quantity','Rate','Discount (%)','Amount (MRP)','Batch','Entry Type(consumption or production)','sale_purchase_ledger']
    itemPart = pd.DataFrame(columns=rb_iv_itemPart)

rb_gst = pd.DataFrame(lst_gst)
if len(lst_gst) > 0:
    rb_gst.columns = rb_gst.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_gst_columns = ['v_sl_no','item_s_no','Ledger Ref No', 'GST Ledger name', 'Rate', 'IGST', 'CGST', 'SGST', 'Cess','Amount Type (cr/dr)']
    rb_gst = pd.DataFrame(columns=rb_iv_gst_columns)

accountingpart = pd.DataFrame(lst_round)
if len(lst_round) > 0:
    accountingpart.columns = accountingpart.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_accountingPart = ['v_sl_no','ledger_name','dr','cr','Ledger Ref No']
    accountingpart =  pd.DataFrame(columns=rb_iv_accountingPart)

rb_bills = pd.DataFrame(lst_bills)
if len(lst_bills) > 0:
    rb_bills.columns = rb_bills.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_bill = ['v_sl_no','Bill Ref No', 'Bill Date','Bill Amount','Is Against Ref (y/n)']
    rb_bills =  pd.DataFrame(columns=rb_iv_bill)

rb_invoice = pd.DataFrame(lst_inv)
if len(lst_inv) > 0:
    rb_invoice.columns = rb_invoice.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_invoice = ['Sl No','Invoice Template Name','Key','Value','Voucher Sl No','Sl No','Invoice Template Name','Voucher Sl No','Row_No','Desc','Amt']
    rb_invoice =  pd.DataFrame(columns=rb_iv_invoice)

rb_cc = pd.DataFrame(lst_cc)
if len(lst_cc) > 0:
    rb_cc.columns = rb_cc.columns.str.replace(r"[3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_cc = ['Sl No','Voucher Sl No','Cost Centre Date','Cost Centre / Cost Category Block 1','Cost Centre Flag 1','Cost Centre / Cost Category Block 2','Cost Centre Flag 2','Amount','Ledger Type (Sale/Purchase/Party)']
    rb_cc = pd.DataFrame(columns=rb_iv_cc)

rb_batchSpec = pd.DataFrame(lst_batchSpec)
if len(lst_batchSpec) > 0:
    rb_batchSpec.columns = rb_batchSpec.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_batchSpec = ['v_sl_no','item_s_no','batch_name','specification_name','value']
    rb_batchSpec = pd.DataFrame(columns=rb_iv_batchSpec)

rb_multiDisc = pd.DataFrame(lst_multiDisc)
if len(lst_multiDisc) > 0:
    rb_multiDisc.columns = rb_multiDisc.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_multiDisc = ['v_sl_no','item_s_no','Discount Ledger','Basis(p/f)','Amount']
    rb_multiDisc = pd.DataFrame(columns=rb_iv_multiDisc)

rb_voucherProp = pd.DataFrame(lst_voucherProp)
if len(lst_voucherProp) > 0:
    rb_voucherProp.columns = rb_voucherProp.columns.str.replace(r"[1,2,3,4,5,6,7,8,9,0]", "",regex=True)
else:
    rb_iv_voucherProp = ['Voucher Sl No','Key','Value']
    rb_voucherProp = pd.DataFrame(columns=rb_iv_voucherProp)

# data=pd.concat([data,rdata],ignore_index=True)
s_code = {'01':'Jammu & Kashmir', '02' : 'Himachal Pradesh', '03': 'Punjab', '04':'Chandigarh', '05':"Uttarakhand", "06":'Haryana','07':'Delhi','08':'Rajasthan','09':'Uttar Pradesh','10':'Bihar','11':'Sikkim','12':'Arunachal Pradesh','13':'Nagaland','14':'Manipur','15':'Mizoram','16':'Tripura','17':'Meghalaya','18':'Assam','19':'West Bengal','20':'Jharkhand','21':'Odisha','22':'Chattisgarh','23':'Madhya Pradesh','24':'Gujarat','25':'Daman & Diu','26':'Dadra & Nagar Haveli','27':'Maharashtra','28':'Andhra Pradesh(Old)','29':'Karnataka','30':'Goa','31':'Lakshwadeep','32':'Kerala','33':'Tamil Nadu','34':'Pondicherry','35':'Andaman & Nicobar Islands','36':'Telangana','37':'Andhra Pradesh(New)'}
ldata = data[:]
ldata.rename(columns={'Customer/ Vendor GSTIN': 'Recipient GSTIN', 'ledger_name': 'party_ledger',"Invoice_No":"Voucher Number"}, inplace=True)
ldata.drop_duplicates(['party_ledger'],inplace=True)
ldata.reset_index(drop=True,inplace=True)
ldata=ldata.fillna("")
ldata['ledger_group'] = 'Debtors - Service Cars - RI'+ldata['Segment'].astype(str)+" /"+ldata['Dealer Code'].astype(str)
ldata['GSTIN'] = ldata["Customer GSTIN Number"].replace(0,'')
ldata['state'] = np.where(ldata["GSTIN"]=='','Karnataka',(ldata["GSTIN"].astype(str).str[:2].replace(s_code)))
ldata['GST Registration Type'] =np.where(ldata["GSTIN"] == '',"Unregistered Person","Registered Person") 
ldata['Registration Nature'] = np.where(ldata["GSTIN"] == '',"","Regular") 
ldata['sl_no'] = ldata.index+1
ldata['pan_no'] = np.where(ldata["GSTIN"]=='0',"",ldata["GSTIN"].astype(str).str[2:12])
ldata['address']=''.replace('0','')
#RealBooks Ledger Master Columns
vm_ledger = ["Sl No","Ledger Group Name","Ledger Name",'Ledger Code',"Enable Cost Centre (y/n)","Enable Bill (y/n)","Credit Period (in days)", "SAC Code",'contact_person','Print Name','address1','address2','address3','party_type','location','address','city','state','pincode','vatno','tinno','panno','email','phn','fax','service_taxno','cstno','eccno','range','division','commissionerate','GST Registration Type','Registration Nature','gstin']
# Creating Ledger Master Object
ledger = pd.DataFrame(columns=vm_ledger)

# #Ledger Master
ledger['Sl No']= ldata['sl_no']
ledger['Ledger Group Name'] = ldata['ledger_group']
ledger['partyTypeName'] = ''
ledger['Ledger Name'] = ldata['party_ledger']
ledger['Ledger Code'] = ''
ledger['Enable Cost Centre (y/n)'] = "n"
ledger['Enable Bill (y/n)'] = 'y'
ledger['Credit Period (in days)'] = ""
ledger['contact_person'] = ldata['party_ledger']
ledger['Print Name'] = ''
ledger['Address'] = ''
ledger['state'] = ldata['state'].replace('0',"Karnataka")
ledger['GST Registration Type'] = ldata['GST Registration Type']
ledger['Registration Nature'] = ldata['Registration Nature']
ledger['gstin'] = ldata['GSTIN']
ledger["pan"] = ldata['pan_no'].replace(0,"")
ledger["email"] = ''
ledger["phn"] = ""
ledger["pinCode"] = ""
ledger["city"] = ''
ledger = ledger.fillna("")

#apiItemMasterData
#preparingItemMasterTable
imaster = data[data["Item_name"]!="0"]
imaster = imaster.drop_duplicates(["Item_name"])
# imaster = imaster[~((imaster['Item group']=='Vehicles') & (imaster['Part Category']==0))]
imaster.reset_index(drop=True,inplace=True)
imaster['sl_no'] = imaster.index + 1 

rb_imaster_columns = ['sl_no', 'item_name', 'Item Group', 'unit', 'item_mark', 'excise_no', 'item_code', 'item_barcode', 'batch_enable (y/n)', 'bom_required (y/n)', 'allow_negative_stock (y/n)', 'effect_in_stock (y/n)', 'HSN/SAC', 'service_item (y/n)', 'Include_Tax(y/n)']
stock_item = pd.DataFrame(columns = rb_imaster_columns)
stock_item['sl_no'] = imaster['sl_no']
stock_item['item_name'] =  imaster["Item_name"].astype(str)
stock_item['Item Group'] = 'Spare Parts & Accessories'
stock_item['unit'] = 'Nos'
stock_item['item_mark'] = ""
stock_item['excise_no'] = ""
stock_item['item_code'] = ""
stock_item['item_barcode'] = ""
stock_item['batch_enable (y/n)'] = "n"
stock_item['bom_required (y/n)'] = "n"
stock_item['allow_negative_stock (y/n)'] = "n"
stock_item['effect_in_stock (y/n)'] = "n"
stock_item['HSN/SAC'] =  imaster['Item_name'].astype(int).astype(str)
stock_item['service_item (y/n)'] ="n"
stock_item['Include_Tax(y/n)'] ="n"
stock_item['CGST Rate'] = np.where(imaster['CGST %']>0,imaster['CGST %'],(imaster['IGST %']/2).round(2))
stock_item['SGST Rate'] = np.where(imaster['CGST %']>0,imaster['CGST %'],(imaster['IGST %']/2).round(2))
stock_item['IGST Rate'] = np.where(imaster['IGST %']>0,imaster['IGST %'],(imaster['CGST %']*2).round(2))
stock_item['cessRate'] = ""
stock_item['cessAppDate'] = ""
stock_item['altUnitName1'] = ""
stock_item['altUnitValue1'] = ""
stock_item['altUnitName2'] = ""
stock_item['altUnitValue2'] = ""
stock_item=stock_item.replace('0','')

# Writing Voucher to excel
uFile = r'Integration.xlsx'

writer = pd.ExcelWriter(uFile, engine='xlsxwriter',date_format='dd-mm-yyyy',datetime_format='dd-mm-yyyy')
voucher.to_excel(writer,sheet_name='voucher',index=False)
itemPart.to_excel(writer,sheet_name='itemPart',index=False)
accountingpart.to_excel(writer,sheet_name='accountingPart',index=False)
rb_invoice.to_excel(writer,sheet_name='Invoice',index=False)
rb_gst.to_excel(writer,sheet_name='GST',index=False)
rb_bills.to_excel(writer,sheet_name='billReference',index=False)
rb_cc.to_excel(writer,sheet_name='costCentre',index=False)
rb_batchSpec.to_excel(writer,sheet_name='batchSpec',index=False)
rb_multiDisc.to_excel(writer,sheet_name='multiDisc',index=False)
rb_voucherProp.to_excel(writer,sheet_name='voucherProp',index=False)
writer.close()

ledger.to_excel(r"Ledger.xlsx",index=False)
stock_item.to_excel(r"item.xlsx",index=False)